In [ ]:
%load_ext autoreload
%autoreload 2

import os, sys, mne, json, h5py, pymatreader, re, time, gc;
import globvars as gv
import utils
import utils_tSNE as utsne
import utils_preproc as upre
import matplotlib.pyplot as plt
import numpy as np
import multiprocessing as mpr
import matplotlib as mpl
import scipy.signal as sig
import pandas as pd 
import utils_postprocess_HPC as postp

data_dir = gv.data_dir
from os.path import join as pjoin

from dateutil import parser
from utils_postprocess_HPC import collectCalcResults

In [ ]:
from utils_postprocess_HPC import collectPerformanceInfo3
prefixes_to_collect = ['onlyH_act_only25']

start_time = parser.parse("20 jan 2023 09:00:00")
#end_time = parser.parse("30 Oct 2021 21:21:45")
end_time = parser.parse("30 Oct 2029 21:21:45")

sources_type = 'parcel_aal'  # or ''
load = False # whether I want to do actual load or just collect filenames (for deferred loading)
#groupings_to_collect = ['merge_nothing']; interval_sets_to_collect = ['basic']
groupings_to_collect = ['merge_nothing']; isets_to_collect = ['basic']
load = False
ndaysBefore = None
light_only = 1

pairs = []
pairs += [('covmat_entire','per_subj_per_medcond_best_LFP_wholectx_oversample_LFP256_covmat_entire')]
pairs += [('covmat_rest','per_subj_per_medcond_best_LFP_wholectx_oversample_LFP256_covmat_rest')]

pptype2res = {}
dfs = []
for pptype,subdir_cur in pairs:
    r = collectPerformanceInfo3('S01_off',prefixes_to_collect,
                   interval_groupings=groupings_to_collect,
                           interval_sets =  isets_to_collect,
             nraws_used='[0-9]+', sources_type = sources_type,
            printFilenames=1,
             ndays_before=ndaysBefore,
             use_main_LFP_chan=1,
              subdir=subdir_cur,
                remove_large_items = 1,
              list_only=0, allow_multi_fn_same_prefix=0,
              use_light_files = light_only, rawname_regex_full=0,
              start_time=start_time,
                end_time=end_time, load=load, ret_df = True)


    df, opr, _, _ =r 
    df['pptype'] = pptype
    
    dfs += [df]
    
    pptype2res[pptype]   = df
    
df_all = pd.concat(dfs)

In [ ]:
from utils_postprocess_HPC import loadSingleRes

In [ ]:
def lbd(row):
    rowd = dict(row)
    r = loadSingleRes(rowd,
        use_light_files=1,
        lighter_light=0,
        remove_large_items=1, in_place = False)
    #print(dict(row)  )
    row['loaded'] = True
    return r


df_all['mcf'] = df_all.apply(lbd,1)
df_all['loaded'] = True

In [ ]:
def lbd(row):
    bacc= row['mcf']['XGB_analysis_versions']['all_present_features']['perf_dict']['perf_aver']['balanced_accuracy']
    return bacc


df_all['bacc'] = df_all.apply(lbd,1)


df_all

In [ ]:
from pathlib import Path
p = df['filename_full'].values[0]
def lbd(p):
    #print(type(p),p)
    parent = Path( p ).parts[-2]
    fn = str( Path( p ).parts[-1] )
    pts = parent.split('_')    
    #return '_'.join(pts[-2:])
    return fn

tmp = df['filename_full'].apply(lbd,1)

In [ ]:
tmp.values[:2]